<a href="https://colab.research.google.com/github/KonradGonrad/PyTorch-deep-learning/blob/main/03_PyTorch_computer_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch Computer Vision

## 0. Computer vision libaries in PyTorch

* [`torchvision`](https://pytorch.org/vision/stable/index.html) - base domain libary for Pytorch computer vision
* `torchvision.datasets` - databases and data loading functions for computer vision
* `torchvision.models` - get pretrained computer vision models that can leverage your own problems
* `torchvision.transforms` - functions for manipulating your vision data (images) to be suitable for use with and ML model
* `torch.utils.data.Dataset` - Base dataset class for PyTorch
* `torch.utils.data.Dataloader` - Creates a Python iterable over a dataset


In [ ]:
# Impoprt PyTorch
import torch
from torch import nn

# Import torchvision
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

# Import matplotlib
import matplotlib.pyplot as plt

# Import pandas (maybe will be needed)
import pandas as pd

# Check version
print(torch.__version__)
print(torchvision.__version__)

## 1. Getting a dataset

FashionMNIST is a dataset of greyscale images of clothingf

In [ ]:
train_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor(),
    target_transform=None
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
    target_transform=None
)

In [ ]:
print(f"length of training data: {len(train_data)}")
print(f"length of testing data: {len(test_data)}")

In [ ]:
data, label = train_data[0]
print(data.shape, label)

In [ ]:
target_classes = train_data.classes
target_classes

In [ ]:
classes_to_idx = train_data.class_to_idx
classes_to_idx

In [ ]:
train_data.targets

### 1.1 Check Input and output shapes of data

In [ ]:
print(f"data of shape {data.shape} -> [Color, Height, Weight] equals {target_classes[label]}")

### 1.2 Visualizing our data


In [ ]:
torch.manual_seed(42) # Setting up a random seed to have same outputs
fig = plt.figure(figsize=(9, 9)) # Creating a figure, where we'll put our subplots
rows, cols = [4, 4] # Setting up dimension, there is 4x4 what gives us 16 pics
for i in range(1, rows*cols + 1): # Setting up loop to put our imgs into figure
  random_idx = torch.randint(0, len(train_data), size=[1]).item() # Getting random index from 0 to train_data max index
  img, label = train_data[random_idx] # assign img and label to variables
  fig.add_subplot(rows, cols, i) # Adding subplot to figure at i index
  plt.imshow(img.squeeze(), cmap='gray') # Creating image plot which is added to figure
  plt.title(target_classes[label]) # Adding title
  plt.axis(False) # Removing aaxis, beacouse it's useless in our case

Do you think these items of clothing (images) could be modelled with pure linear lines? Or do you think we'll need non-linearities

## 2. Prepare dataloader

Right now, our data is in the form of PyTorch datasets

DataLoader turns our dataset into Python iterable

More specifically, we want to turn our data into batches (or mini-batches).

Why would we do this?

1. It is more computationally efficient, as in, your computing hardware may not be able to look (store in memory) at 60000 images in one hit. So we break it down to 32 images at a time (batch size of 32)
2. It gives our neural network more chances to update its gradients per epoch

for more on [mini-batches](https://www.bilibili.com/video/BV1RE411Z7YW/)

In [ ]:
train_data, test_data

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

test_dataloader = DataLoader(dataset=test_data,
                             batch_size=BATCH_SIZE,
                             shuffle=False)

train_dataloader, test_dataloader

In [ ]:
print(f"train dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}...")
print(f"test dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}...")

In [ ]:
train_features_batch, train_labels_batch = next(iter(train_dataloader))
print(f"train_feature_batch shape: {train_features_batch.shape} and train_labels_batch shape: {train_labels_batch.shape}")

In [ ]:
random_idx = torch.randint(0, len(train_features_batch), size=[1]).item()
img, title = train_features_batch[random_idx], train_labels_batch[random_idx]
plt.imshow(img.squeeze(), cmap='gray')
plt.title(target_classes[label])
plt.axis(False)
print(f"image shape: {img.shape}")
print(f"Label: {label}, label size: {title.shape}")

## 3.0 Model 0: Build a baseline model

In [ ]:
# Create a flatten layer
flatten_layer = nn.Flatten()

# Get a single sample
x = train_features_batch[0]
x

# Flatten the sample
output = flatten_layer(x)
output

In [ ]:
from torch import nn
class FashionMNISTModelV0(nn.Module):
  def __init__(self,
               input_features: int,
               hidden_layers: int,
               output_features: int):
    super().__init__()
    self.layer_1 = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=input_features, out_features=hidden_layers),
        nn.Linear(in_features=hidden_layers, out_features=output_features)
    )

  def forward(self, x):
    return self.layer_1(x)




In [ ]:
model_0 = FashionMNISTModelV0(input_features=784,
                              hidden_layers=10,
                              output_features=len(target_classes))

In [ ]:
model_0(x).shape

In [ ]:
x_dummy = torch.rand([1, 1, 28, 28])
model_0(x_dummy).shape

In [ ]:
model_0.state_dict()

### 3.1 Setup loss function, optimizer and evaluation metrics

* Loss Function - since we're working with multi-class data, our loss function will be 'nn.CrossEntropyLoss()'
* Optimizer - our optimizer 'torch.optim.SGD()' (stochastic gradient descent)
* Evaluation metric - since we're working on a classification problem, let's use accuracy as our evaluation metric



In [ ]:
from pathlib import Path
import requests

if Path('helper_functions.py').is_file():
  print('helper_functions.py is already downloaded')
else:
  print("Downloading helper_functions.py")
  request = requests.get('https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py')
  with open('helper_functions.py', 'wb') as f:
    f.write(request.content)

In [ ]:
# Import accuracy_fn
from helper_functions import accuracy_fn

# Loss fn and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_0.parameters(),
                            lr=0.1)

### 3.2 Creating a function to time our experiments

Machine learning is very experimental.

Two of the main thing you'll often want to track are:
1. Model's performance(loss and accuracy values etc)
2. How fast it runs

In [ ]:
from timeit import default_timer as timer

def time_start_end(start_time: float,
                   end_time: float,
                   device: torch.device = None):
  total_time = end_time - start_time
  print(f"Took about {total_time:.3f} seconds to execute the code on {device}")
  return total_time

In [ ]:
start = timer()
# break
end = timer()

time_start_end(start_time=start,
               end_time=end,
               device='cpu')

### 3.3 Creating a training loop and training a model on batches of data

1. Loop through epochs.
2. Loop through training batches, perform training steps, calculate the train loss *per batch*
3. Loop through testing batches, perform testing steps, calculate the test loss *per batch*.
4. Print out what's happening.
5. Time it all

In [ ]:
# Import tqdm for progress bar
from tqdm.auto import tqdm

# Set the seed and start the timer
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
train_loop_start_time = timer()

# Set the number of epochs(we'll keep this small for faster training time)
epochs = 3

# Create training and test loop
for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}\n-----")
  #Training
  train_loss, train_acc = 0, 0
  # Add a loop to loop thorught the training batches
  for batch, (X, y) in enumerate(train_dataloader):
    model_0.train()
    # 1. Forward pass
    y_pred = model_0(X)

    # 2. Calculate loss
    loss = loss_fn(y_pred, y)
    train_loss += loss

    acc = accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))
    train_acc += acc

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4.Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    # Print out what's happening
    if batch % 400 == 0:
      print(f"Progress: {batch * len(X)}/{len(train_dataloader.dataset)} samples")

  # divide total train loss by length of train dataloader
  train_loss /= len(train_dataloader)
  train_acc /= len(train_dataloader)

  ### Testing
  test_loss, test_acc = 0, 0
  model_0.eval()
  with torch.inference_mode():

    for X, y in test_dataloader:
      # 1. Forward pas
      test_pred = model_0(X)

      # 2. Calculate the loss
      test_loss += loss_fn(test_pred, y)

      # 3. Calculate accuracy
      test_acc += accuracy_fn(y_true=y, y_pred=test_pred.argmax(dim=1))

    # Calculate the test loss average per batch
    test_loss /= len(test_dataloader)

    # Calculate the test acc average per batch
    test_acc /= len(test_dataloader)

  # Print out what's happening
  print(f"Train_loss: {train_loss:.5f}, Train_acc: {train_acc:.5f} | Test_loss: {test_loss:.5f}, Test_acc: {test_acc:.5f}")

# Calculate training time
train_loop_end_time = timer()
total_train_time_model_0 = time_start_end(start_time=train_loop_start_time,
               end_time=train_loop_end_time,
               device=str(next(model_0.parameters()).device))


In [ ]:
next(model_0.parameters()).device

### 4. Make predictions and get model 0 results

In [ ]:
torch.manual_seed(42)

def eval_mode(data: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              acc_fn,
              device: torch.device
              ):
  loss, acc = 0, 0
  model.eval()
  with torch.inference_mode():
    for X, y in tqdm(data):
      X, y = X.to(device), y.to(device)
      y_pred = model(X)

      loss += loss_fn(y_pred, y)
      acc += acc_fn(y, y_pred.argmax(dim=1))
    loss /= len(data)
    acc /= len(data)
  return {"model": model.__class__.__name__,
          "loss": round(loss.item(), 2),
          "acc": round(acc, 2)}

In [ ]:
model_0_results = eval_mode(data=test_dataloader,
          model = model_0,
          loss_fn=loss_fn,
          acc_fn=accuracy_fn,
          device='cpu')

## 5. Setup device agnostic-code (for using a gpu if there is one)


In [ ]:
# Device agnostic code
DEVICE_DESTINATION = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE_DESTINATION

In [ ]:
if torch.cuda.is_available():
  !nvidia-smi
else:
  print('no gpu found')

## 6. Model 1: Building a better model with non-linearity

We learned about the power of non-linearity

In [ ]:
class FashionMNISTModelV1(nn.Module):
  def __init__(self,
               input_layers: int,
               hidden_layers: int,
               output_layers: int):
    super().__init__()
    self.layer_1 = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=input_layers, out_features=hidden_layers),
        nn.ReLU(),
        nn.Linear(in_features=hidden_layers, out_features=output_layers ),
        nn.ReLU()
    )

  def forward(self, x):
    return self.layer_1(x)

In [ ]:
model_1 = FashionMNISTModelV1(input_layers=784,
                              hidden_layers=10,
                              output_layers=len(target_classes)).to(DEVICE_DESTINATION)
model_1

In [ ]:
next(model_1.parameters()).device

### 6.1 Setup loss and evaluation metrics

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_1.parameters(),
                            lr=0.1)

### 6.2 Functionizing training and evaluation/testing loops
Let's create a function for:
* training loop - train_step()
* testing loop - test_step

In [ ]:
def train_step(model: torch.nn.Module,
               data: torch.utils.data.DataLoader,
               loss_fn: torch.nn,
               acc_fn: any,
               optimizer: torch.optim,
               device: torch.device = DEVICE_DESTINATION):
  """Performs a training loop step on model going over data_loader"""
  train_loss, train_acc = 0, 0
  model.train()
  for batch, (X, y) in enumerate(data):
    # 0. Data on target device
    X, y = X.to(device), y.to(device)
    # 1. forward pass
    y_pred = model(X) # Logits

    # 2. Calculate the loss and accuracy
    loss = loss_fn(y_pred, y)
    acc = acc_fn(y, y_pred.argmax(dim=1))

    train_loss += loss
    train_acc += acc

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Optimzizer step step step
    optimizer.step()
  train_loss /= len(data)
  train_acc /= len(data)

  print(f"Train loss: {train_loss:.4f} | Train acc: {train_acc:.4f}%\n")

In [ ]:
def test_step(model: torch.nn.Module,
              data: torch.utils.data.DataLoader,
              acc_fn: any,
              loss_fn: torch.nn,
              device: torch.device = DEVICE_DESTINATION):
  """Performs a testing loop step on model going over data_loader"""
  model.eval()
  loss, acc = 0, 0
  with torch.inference_mode():
    for X, y in data:
      # 0. Data into device
      X, y = X.to(device), y.to(device)
      # 1. Forward pass
      y_pred = model(X)

      # 2. Calculate the loss and acc
      loss += loss_fn(y_pred, y)
      acc += acc_fn(y_pred.argmax(dim=1), y)

    loss /= len(data)
    acc /= len(data)
    print(f"Test loss: {loss:.4f} | Test acc: {acc:.4f}%\n")

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

from timeit import default_timer as timer
train_time_start_on_gpu = timer()

epochs = 3
for epoch in tqdm(range(epochs)):
  print(f'Epoch {epoch}\n -----')
  train_step(model=model_1,
             data=train_dataloader,
             loss_fn=loss_fn,
             acc_fn=accuracy_fn,
             optimizer=optimizer,
             device=DEVICE_DESTINATION)

  test_step(model=model_1,
            data=test_dataloader,
            acc_fn=accuracy_fn,
            loss_fn=loss_fn,
            device=DEVICE_DESTINATION)

train_time_end_on_gpu = timer()
total_train_time_model_1 = time_start_end(start_time = train_time_start_on_gpu,
               end_time=train_time_end_on_gpu,
                                          device=next(model_1.parameters()).device)

In [ ]:
print(f"GPU time: {total_train_time_model_1} seconds")

In [ ]:
print(f"GPU time: {total_train_time_model_0} seconds")

In [ ]:
model_1_results = eval_mode(data=test_dataloader,
                            model=model_1,
                            loss_fn=loss_fn,
                            acc_fn=accuracy_fn,
                            device='cuda')

In [ ]:
model_0_results

In [ ]:
model_1_results